In [ ]:
import requests
import json
import torch
from diffusers import StableDiffusionPipeline
from PIL import Image
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import Layout

# OpenAI API Key
openai_api_key = "enter your api key here"
# Input from user CLI
user_input_widget = widgets.Textarea(
    value='',
    placeholder='Describe something you want enhanced (e.g., "a tree in the forest")',
    description='Your Description:',
    disabled=False,
    layout=Layout(width="80%", height="100px")
)

#  CLI for enhanced text
enhanced_text_box = widgets.HTML(
    value="",
    placeholder='Enhanced text will appear here.',
    layout=Layout(width="80%", border="1px solid gray", padding="10px", margin="10px 0")
)

#title of the system
title = widgets.HTML(
    value="<h2 style='color:#4CAF50; font-family: Arial;'>Enter Something That you want to see 20 years from now!</h2>"
)

# basic progress bar implemented
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Loading:',
    bar_style='info',
    orientation='horizontal',
    layout=Layout(width="80%", visibility="hidden")
)

#  output CLI
output_area = widgets.Output(layout={'border': '1px solid gray', 'padding': '10px', 'margin': '10px 0'})

# promt that is given to GPT3.5 Turbo
def generate_image(user_input):
    prompt = f"""
Imagine how {user_input} would look 20 years from now. Describe it in a hyper-realistic, and detailed manner.
Use artistic and visual language as if describing a film scene. Include colors, lighting, mood, atmosphere, and textures.
Limit your response to 50 words, and make it vivid enough to be directly converted into an image.
"""

#connection to apı URL
    url = "https://api.openai.com/v1/chat/completions"
# Content Type is set to JSON in order to print the error message if occurs

    headers = {
        "Authorization": f"Bearer {openai_api_key}",
        "Content-Type": "application/json"
    }
    #data structure goint to be sent to API
    data = {
        "model": "gpt-3.5-turbo",
        "messages": [
            {"role": "system", "content": "You are a creative assistant that generates vivid and realistic descriptions of future scenarios."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.7,
        "max_tokens": 90
    }
#request sent by post method and if succesfull response JSON obtained
    response = requests.post(url, headers=headers, json=data)
    if response.status_code in [200, 201, 203]:
        result = response.json()
        enhanced_text = result["choices"][0]["message"]["content"]
        return enhanced_text
    else:
        with output_area:
            print("JSON COULD NOT OBTAAINED", response.text)
        return None


def generate_image_from_text(enhanced_text):
    if enhanced_text is None:
        return -1

    #  Progress bar CLI
    progress_bar.layout.visibility = 'visible'
    progress_bar.value = 0
#ensuring that whether cpu or gpu going to be envolved
    model_id = "runwayml/stable-diffusion-v1-5"
    device = "cuda" if torch.cuda.is_available() else "cpu"
    #enhance text is updated to obtain better image
    enhanced_text += ", cinematic, hyper-realistic, 8K, intricate lighting, dramatic shadows"

    #  Stable Diffusion pipeline
    pipe = StableDiffusionPipeline.from_pretrained(
        model_id,
        torch_dtype=torch.float16 if device == "cuda" else torch.float32
    ).to(device)

    progress_bar.value = 50

    #  image generated
    image = pipe(enhanced_text).images[0]
    progress_bar.value = 100

    with output_area:
        output_area.clear_output()
        display(image)


    progress_bar.layout.visibility = 'hidden'

#  Buton CLI
def on_button_click(b):
    output_area.clear_output()
    enhanced_text_box.value = ""
    user_input = user_input_widget.value.strip()

    if user_input:
        enhanced_text = generate_image(user_input)

        if enhanced_text:
           formatted_text = enhanced_text.replace('\n', '<br>')
           enhanced_text_box.value = f"<b> Enhanced Description:</b><br>{formatted_text}"
           generate_image_from_text(enhanced_text)
    else:
        with output_area:
            print("please enter a description")

#  Buton CLI
generate_button = widgets.Button(
    description="Generate Image",
    layout=Layout(width="200px", height="40px", margin="10px 0"),
    button_style='primary'
)
generate_button.on_click(on_button_click)

#  UI implementation
ui = widgets.VBox([
    title,
    user_input_widget,
    generate_button,
    enhanced_text_box,
    progress_bar,
    output_area
])

display(ui)


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (83 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: [', intricate lighting , dramatic shadows']


  0%|          | 0/50 [00:00<?, ?it/s]